In [2]:
import pandas as pd

In [3]:
df =  pd.read_csv('dataNew_df.csv')

In [5]:
target_column = 'target'


X = df.drop(columns=[target_column])
y = df[target_column]

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, KFold

def train_model(X, y):
    """
    Run random forest classification model on feature subset
    and retrieve cross validated ROC-AUC score
    """
    clf = RandomForestClassifier(random_state=42)
    kf = KFold(shuffle=True, n_splits=3, random_state=42)
    cv_roc_auc_score = round(cross_val_score(clf, X, y.values.ravel(), cv=kf, 
                                             scoring="roc_auc", n_jobs=-1).mean(), 3)

    return cv_roc_auc_score

In [10]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import StratifiedKFold

def simulated_annealing(X, y, maxiters=50, alpha=0.85, beta=1, T_0=1, update_iters=1, temp_reduction='geometric', n_splits=5):
    """
    Function to perform feature selection using simulated annealing with cross-validation.
    Inputs:
    X: Predictor features
    y: Labels
    maxiters: Maximum number of iterations
    alpha: Factor to reduce temperature
    beta: Constant in probability estimate
    T_0: Initial temperature
    update_iters: Number of iterations required to update temperature
    temp_reduction: Strategy for temperature reduction schedule
    n_splits: Number of cross-validation splits

    Output:
    1) Dataframe of parameters explored and corresponding model performance
    2) Average best metric score (i.e., AUC score in this case) across folds
    3) List of subset features that correspond to the best metric
    """
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    columns = ['Fold', 'Iteration', 'Feature Count', 'Feature Set',
               'Metric', 'Best Metric', 'Acceptance Probability',
               'Random Number', 'Outcome']
    results = pd.DataFrame(columns=columns)
    best_subset = None
    hash_values = set()
    T = T_0

    for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
        X_train, X_val = X.iloc[train_index, :], X.iloc[val_index, :]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        # Get ascending range indices of all columns
        full_set = set(np.arange(len(X_train.columns)))

        # Generate initial random subset based on ~50% of columns
        curr_subset = set(random.sample(list(full_set), round(0.5 * len(full_set))))

        # Get baseline metric score (i.e. AUC) of initial random subset
        X_curr = X_train.iloc[:, list(curr_subset)]
        prev_metric = train_model(X_curr, y_train)
        best_metric = prev_metric

        for i in range(maxiters):
            # Termination conditions
            if T < 0.01:
                print(f'Temperature {T} below threshold. Termination condition met')
                break

            print(f'Starting Iteration {i + 1}')

            # Execute perturbation (i.e., alter current subset to get a new subset)
            while True:
                # Decide what type of perturbation to make
                if len(curr_subset) == len(full_set):
                    move = 'Remove'
                elif len(curr_subset) == 2:  # Not to go below 2 features
                    move = random.choice(['Add', 'Replace'])
                else:
                    move = random.choice(['Add', 'Replace', 'Remove'])

                # Get columns not yet used in the current subset
                pending_cols = full_set.difference(curr_subset)
                new_subset = curr_subset.copy()

                if move == 'Add':
                    new_subset.add(random.choice(list(pending_cols)))
                elif move == 'Replace':
                    new_subset.remove(random.choice(list(curr_subset)))
                    new_subset.add(random.choice(list(pending_cols)))
                else:
                    new_subset.remove(random.choice(list(curr_subset)))

                if new_subset in hash_values:
                    print('Subset already visited')
                else:
                    hash_values.add(frozenset(new_subset))
                    break

            # Filter dataframe to the current subset
            X_new = X_train.iloc[:, list(new_subset)]

            # Get metric of the new subset
            metric = train_model(X_new, y_train)

            if metric > prev_metric:
                print('Local improvement in metric from {:8.4f} to {:8.4f} '
                      .format(prev_metric, metric) + ' - New subset accepted')
                outcome = 'Improved'
                accept_prob, rnd = '-', '-'
                prev_metric = metric
                curr_subset = new_subset.copy()

                # Keep track of overall best metric so far
                if metric > best_metric:
                    print('Global improvement in metric from {:8.4f} to {:8.4f} '
                          .format(best_metric, metric) + ' - Best subset updated')
                    best_metric = metric
                    best_subset = new_subset.copy()

            else:
                rnd = np.random.uniform()
                diff = prev_metric - metric
                accept_prob = np.exp(-beta * diff / T)

                if rnd < accept_prob:
                    print('New subset has worse performance but still accept. Metric change' +
                          ':{:8.4f}, Acceptance probability:{:6.4f}, Random number:{:6.4f}'
                          .format(diff, accept_prob, rnd))
                    outcome = 'Accept'
                    prev_metric = metric
                    curr_subset = new_subset.copy()
                else:
                    print('New subset has worse performance, therefore reject. Metric change' +
                          ':{:8.4f}, Acceptance probability:{:6.4f}, Random number:{:6.4f}'
                          .format(diff, accept_prob, rnd))
                    outcome = 'Reject'

            # Update results dataframe
            results.loc[i, 'Fold'] = fold + 1
            results.loc[i, 'Iteration'] = i + 1
            results.loc[i, 'Feature Count'] = len(curr_subset)
            results.loc[i, 'Feature Set'] = sorted(curr_subset)
            results.loc[i, 'Metric'] = metric
            results.loc[i, 'Best Metric'] = best_metric
            results.loc[i, 'Acceptance Probability'] = accept_prob
            results.loc[i, 'Random Number'] = rnd
            results.loc[i, 'Outcome'] = outcome

            # Temperature cooling schedule
            if i % update_iters == 0:
                if temp_reduction == 'geometric':
                    T = alpha * T
                elif temp_reduction == 'linear':
                    T -= alpha
                elif temp_reduction == 'slow decrease':
                    b = 5  # Arbitrary constant
                    T = T / (1 + b * T)
                else:
                    raise Exception("Temperature reduction strategy not recognized")

        

    # Convert column indices of the best subset to original names
    best_subset_cols = [list(X.columns)[i] for i in list(best_subset)]

    # Drop NaN rows in results
    results = results.dropna(axis=0, how='all')

    return results, results['Best Metric'].mean(), best_subset_cols


In [11]:
results, best_metric, best_subset_cols = simulated_annealing(X, y)

Starting Iteration 1
New subset has worse performance but still accept. Metric change:  0.0950, Acceptance probability:0.9094, Random number:0.1726
Starting Iteration 2
New subset has worse performance but still accept. Metric change:  0.1530, Acceptance probability:0.8353, Random number:0.6740
Starting Iteration 3
Local improvement in metric from   0.3240 to   0.6090  - New subset accepted
Global improvement in metric from   0.5720 to   0.6090  - Best subset updated
Starting Iteration 4
New subset has worse performance but still accept. Metric change:  0.1160, Acceptance probability:0.8279, Random number:0.3274
Starting Iteration 5
New subset has worse performance but still accept. Metric change:  0.0950, Acceptance probability:0.8336, Random number:0.2884
Starting Iteration 6
New subset has worse performance but still accept. Metric change:  0.1640, Acceptance probability:0.6910, Random number:0.5023
Starting Iteration 7
Local improvement in metric from   0.2340 to   0.4650  - New su

In [12]:
len(best_subset_cols)

7202

In [13]:
best_subset_cols


['rna_NUTF2',
 'rna_TRIM6',
 'rna_PAX7',
 'rna_PSTPIP1',
 'rna_GTF2IRD1',
 'rna_PTRHD1',
 'rna_DNAJC4',
 'rna_CIITA',
 'rna_LINC00260',
 'rna_VAX2',
 'rna_MYO5C',
 'rna_TPPP3',
 'rna_SF3B2',
 'rna_LGALS13',
 'rna_OGFRL1',
 'rna_SECISBP2',
 'rna_LINC01356',
 'rna_SEMA4G',
 'rna_RNASE4',
 'rna_CIAPIN1',
 'rna_ST3GAL5',
 'rna_TTC28',
 'rna_ZCCHC11',
 'rna_PLD2',
 'rna_LOC100134868',
 'rna_CACNG8',
 'rna_EN1',
 'rna_PON3',
 'rna_NUTM2A-AS1',
 'rna_TM2D1',
 'rna_CALM3',
 'rna_AGAP4',
 'rna_NEIL3',
 'rna_PROM2',
 'rna_GEMIN8P4',
 'rna_NUDT16',
 'rna_ROBO3',
 'rna_ANKRD40',
 'rna_TTC14',
 'rna_DIP2B',
 'rna_TMEM9B',
 'rna_LOC101929709',
 'rna_PSPH',
 'mut_HTR3E',
 'rna_PPP1R1A',
 'rna_SPG7',
 'rna_SND1-IT1',
 'rna_LOC399815',
 'rna_HIST1H1C',
 'rna_SPINK5',
 'mut_DMXL2',
 'rna_BLM',
 'rna_PRPF38B',
 'rna_YPEL3',
 'rna_CRYM',
 'rna_SPR',
 'rna_ZNF542P',
 'rna_ZIC2',
 'rna_HPDL',
 'rna_ICOSLG',
 'rna_AJAP1',
 'rna_ARID1B',
 'rna_TMEM189-UBE2V1',
 'rna_ERGIC2',
 'rna_ARHGAP31-AS1',
 'rna_PRR13',

In [16]:
Selected_df = df[best_subset_cols + ['target']]

In [19]:
Selected_df.head()

rna_NUTF2  rna_TRIM6  rna_PAX7  rna_PSTPIP1  rna_GTF2IRD1  rna_PTRHD1  \
0   4.934960   0.945073  0.000000     0.000000      2.095631    2.689632   
1   4.923149   0.635339  0.229926     0.112581      1.132597    3.209151   
2   6.202956   0.000000  0.000000     0.158527      2.746876    4.556957   
3   4.178978   2.923298  0.042255     0.061528      1.430087    6.298699   
4   6.604197   0.551486  0.052740     0.111196      2.042336    4.772530   

   rna_DNAJC4  rna_CIITA  rna_LINC00260  rna_VAX2  ...  rna_HOXB-AS3  \
0    3.088082   0.000000       0.779708  0.330414  ...      1.834982   
1    2.437968   0.081909       0.819465  0.601460  ...      0.000000   
2    2.431198   0.074207       0.695231  0.993611  ...      0.000000   
3    0.364494   0.079786       0.398892  0.407078  ...      0.000000   
4    4.339779   0.046760       0.248016  1.099484  ...      0.312550   

   rna_LOC728613  rna_ZNF611  rna_C11orf54  type_Bowel  type_Breast  \
0       0.385249    0.944978      1.735098           0            1   
1       0.444204    0.781026      0.927757           0            1   
2       1.041019    1.392240      0.967260           0            1   
3       0.510785    1.289498      2.243730           0            1   
4       1.244917    1.176935      3.099765           0            1   

   type_Kidney  type_Lung  type_Ovary  target  
0            0          0           0       1  
1            0          0           0       0  
2            0          0           0       1  
3            0          0           0       0  
4            0          0           0       1  

[5 rows x 7203 columns]